# やること

MatlabのコードをJuliaに翻訳する。

## 準備

In [1]:
using Optim
using StatsFuns
using DataFrames

# inverse beta distribution function を行列に対応するように拡張
import StatsFuns.betainvcdf
betainvcdf(alpha::Number, beta::Number, x::Array) = reshape([betainvcdf(alpha, beta, i) for i in reshape(x, 1, size(x, 1)*size(x, 2)) ], size(x, 1), size(x, 2))

# maxをArray{String, 1}に対応するように拡張
# しかしArray型で入っているのでAnyに対応させる
import Base.max
max(number::Real, comparison::Any) = [max(number, parse(Int, i)) for i in comparison] 

# normpdfを配列に拡張
import StatsFuns.normpdf
normpdf(array::Array{Float64, 2}) = reshape([normpdf(i) for i in reshape(array, 1, size(array, 1)*size(array, 2))], size(array, 1), size(array, 2))

# parameterの初期値を作成
# inivalueは260こ
# learning_paramsは13こ
iii = 1 #kokokaeruiii;
ini = DataFrame(randn(260, 100))
learn = DataFrame(randn(13, 100))
writetable(*("inivalue_", "$iii", ".txt"), ini)
writetable(*("learning_params", "$iii", ".txt"), learn)

## new_loglike2の翻訳

In [2]:
;cat loglike.jl

function new_loglike(param::Array{Float64,1}, DATA::Array{Real,2}, Cand::Array{Int64, 2})
    
    # これなんだかわからないのでコメントアウト
    #if i_bayes == 1
        #param = param
    #end
    
    FAlph = Array(Float64, 2, 1)
    # setting parameters
    param[2] = -1
    C0 = 0
    # Raceの変更に伴い、Cxも4つの要素にする必要がある。
    q = 4
    Cx = param[1:q]
    Cz = -abs(param[q+1:q+3])
    FAlph[1,1] = abs(param[q+4])
    FAlph[2,1] = abs(param[q+5])
    param[q+4:q+5] = FAlph[1:2,1]
    Sig_xsi  = max(0.5, abs(param[q+6]))
    param[q+6] = Sig_xsi
    DeltaO  = 0.6891
    DeltaMO = 0.5366
    # Raceをいじった関係でこのvkを4*4 = 16この要素にしなくちゃいけない
    vk = param[12:27]
    composite = param[75:149]
    Tij = abs(param[150:260])
    
    # new parameters
    # rho_eta = abs(param[261])
    rho_eta = 1
    rho_chi = param[262:265]
    mu_chi = param[266:269]
    chi = param[270:273]
    
    
    # making log likelihood
    # simulated values
    N_T_ij = [0;1;3;6]
    # 上で拡張した関数を使用
    Alpha = betainvcdf(FAlph[1,1],FAlph[2,1]

In [4]:
include("loglike.jl");

## New_bayes2の翻訳 Candの作成

In [7]:
;cat bayes.jl

# define T as num of period
T = 14
bandwidth = 0.05
N_sim = 100
n_eval = 0
I = 4
N_cand = 4
# load iii.txt -ASCII;
# load jjj.txt -ASCII;

# dataのロード
data = readtable("data.csv", header = false)
data = convert(Array, data)

# initial valueのロード
iii = 1
jjj = 1
kkk = 1

inivalue = readtable(*("inivalue_", "$iii", ".txt"))
inivalue = inivalue[:, jjj]

learning_params = readtable(*("learning_params", "$iii", ".txt"))
learning_params = learning_params[:, jjj]

parameter = vcat(1000000000, 0, inivalue, learning_params)
writetable(*("parameter_", "$iii", "_", "$jjj", "_", "$kkk", ".txt"), DataFrame(X = parameter))


#　ここから本題
D　=　data[:,45] .> 100 # Remove municipality with small populaion <100
# 行削除はどうするのか、そもそもいるのか
# data[D, :] = [];
data = data[D, :]

# Construct X : Regressor for individual characteristics
X = [1 1 1 1 1];
dFX = ones(size(data, 1))
Race = [1 0 0; 0 1 0; 0 0 1];
Educ = 0.1*[16;14;12;9];
Incm = log([20000;35000;72500;120000]);

# (white,black,otherasian+indian+other)
dFXRace 

In [5]:
include("bayes.jl");

## 尤度関数が動くことを確認

かなり遅いので最適化以前に尤度の計算をスムーズに行えるようにするべき？

例えば、

- 行列で操作しているところをfor文に変える（あまり効かないっぽい）
- 変数ごとの型を最適化しておく
- Candを事前にやる（やった）

などが考えられる。

部分ごとに時間計測した結果最後のループで1秒ずつぐらいかかってることがわかった。（それ以外の部分は大して負担ではない）

以下実行した改良

- Candの処理（Int64化と並び替え）をfunctionの外で行い直接アクセスできるようにした。→2秒ほど改善
- とりあえずreshapeの回数を減らした。→配列の事前設置がややこしいのでやめ
- for文に書き換える→さらに2,3秒改善(一つfor文を使うと遅くなる処理があったのでそれを除いてfor文化した。また、メインで時間がかかっているループ内のみ書き換えている）

In [6]:
# このセルは残す
# 行列処理のtime(Candを外に出した。)
@time new_loglike(parameter, DATA, Cand)

 38.814163 seconds (262.30 M allocations: 10.311 GB, 5.36% gc time)


349408.00362013566

In [8]:
# このセルは残す
# for文のtime
@time new_loglike(parameter, DATA, Cand)

 36.975457 seconds (492.10 M allocations: 13.481 GB, 9.00% gc time)


322118.23684017756

In [7]:
# このセルは残す
# for文、rho_etaを固定、VSTRの計算を訂正
@time new_loglike(parameter, DATA, Cand)

 34.392904 seconds (490.25 M allocations: 13.414 GB, 9.30% gc time)


110960.00849019532

## New_Bayes2の翻訳　最適化パート

あまりに時間がかかったので途中で中断しました。

In [11]:
result = optimize(new_loglike(parameter, DATA), parameter, BFGS())
theta = Optim.minimizer(result)
likelihood = Optim.minimum(result)

LoadError: LoadError: InterruptException:
while loading In[11], in expression starting on line 1